## Build a News ETL Data Pipeline Using Python and SQLite

### Task 1: Import Libraries and Connect to the News API

In [1]:
import pandas as pd 
from typing import Union
import sqlite3
import newsapi
from newsapi import NewsApiClient
import logging

In [ ]:
news_api = NewsApiClient(api_key="")

### Task 2: Retrieve and Print News Articles

In [3]:
# set up root logger 
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
def extract_news_data(api_client: newsapi.newsapi_client.NewsApiClient) -> list:
    """
        Retrieves english news articles related to AI from the newsAPI: https://newsapi.org/docs/client-libraries/python
        
        Args:
            api_client: the initialised news api client

        Raises:
            Exception if API connection is unsuccessful 
            
        Returns: a list of the retrieved news article
    """
    try:
        api_results = api_client.get_everything(q='AI',
                                                sort_by='publishedAt',
                                                language='en'
                                        )

        logging.info("Connection successful: %s", api_results['status'])
        return api_results['articles']
    except Exception as e:
        logging.error("Connection unsuccessful: %s", e)

In [5]:
articles = extract_news_data(news_api)

2025-07-27 15:51:25,355 - INFO - Connection successful: ok


In [6]:
print(articles[:3])

[{'source': {'id': None, 'name': 'Pypi.org'}, 'author': None, 'title': 'aaip added to PyPI', 'description': 'AI Agent Identity Protocol (AAIP) - Standard delegation format for AI agent authorization', 'url': 'https://pypi.org/project/aaip/', 'urlToImage': 'https://pypi.org/static/images/twitter.abaf4b19.webp', 'publishedAt': '2025-07-26T15:50:53Z', 'content': 'Standard delegation format for AI agent authorization\r\nWhat is AAIP?\r\nAAIP is a standard format for users to grant specific, time-bounded, and constrained permissions to AI agents. It provides crypto… [+6958 chars]'}, {'source': {'id': None, 'name': 'Pypi.org'}, 'author': None, 'title': 'aaip 1.0.0', 'description': 'AI Agent Identity Protocol (AAIP) - Standard delegation format for AI agent authorization', 'url': 'https://pypi.org/project/aaip/1.0.0/', 'urlToImage': None, 'publishedAt': '2025-07-26T15:50:53Z', 'content': 'A required part of this site couldnt load. This may be due to a browser\r\n extension, network issues, or

### Task 3: Clean Author Column

In [ ]:
def clean_author_column(text: str) -> str:
    """
    
    """
    try:
        reformatted_text = text.split(",")[0].title()
        return reformatted_text
    except AttributeError: 
        return 'No Author'

### Task 4: Transform News Data

In [ ]:
def transform_news_data(articles: list) -> pd.DataFrame:
    """ 
    Transforms the data and loads it into a dataframe 

    This function extracts the necessary features from the data and performs some data cleaning 

    Args: 
        the list of articles returned from the API

    Returns:
        a pandas dataframe of the extracted, cleaned data
    """
    article_list = []
    for article in articles:
        author = article['author']
        title = article['title']
        published_at = article['publishedAt']
        content = article['content']
        url = article['url']
        source = article['source']['name']

        article_list.append([source, author, title, url, published_at, content])

    df = pd.DataFrame(data=article_list, 
                    columns=['Source', 'Author Name', "News Title", "URL", "Date Published", "Content"])

    
    df['Date Published'] = pd.to_datetime(df['Date Published'], format='ISO8601')
    df["Author Name"] = df["Author Name"].apply(clean_author_column)
    return df

In [9]:
transformed_df = transform_news_data(articles)
transformed_df.head()

,Source,Author Name,News Title,URL,Date Published,Content
0,Pypi.org,No Author,aaip added to PyPI,https://pypi.org/project/aaip/,2025-07-26 15:50:53+00:00,Standard delegation format for AI agent author...
1,Pypi.org,No Author,aaip 1.0.0,https://pypi.org/project/aaip/1.0.0/,2025-07-26 15:50:53+00:00,A required part of this site couldnt load. Thi...
2,Pypi.org,Mimicx@Speedpresta.Com,mimicx 0.1.50,https://pypi.org/project/mimicx/0.1.50/,2025-07-26 15:48:52+00:00,A required part of this site couldnt load. Thi...
3,Yahoo Entertainment,Gabrielle Olya,AI Could Replace Millions of Jobs: Robert Kiyo...,https://finance.yahoo.com/news/ai-could-replac...,2025-07-26 15:47:09+00:00,As artificial intelligence (AI) continues to a...
4,Khabarhub.com,Khabarhub,Communications Minister asks stakeholders to w...,https://english.khabarhub.com/2025/26/487748/,2025-07-26 15:45:17+00:00,KATHMANDU: Minister for Communications and Inf...


### Task 5: Load the Data into SQLite Database


In [10]:
def load_news_data(data: pd.DataFrame) -> None:
    """
    Connects with the sqlite database, creates a table and inserts the data parameter into the table 

    Args: 
        data: the data to be added to the database
    
    Returns:
        None 

    Raises:
        Exception: Catches and logs any general exception that occurs during database connection, table creation,
                   or data insertion (e.g., sqlite3.Error, pandas-related errors).
    """
    database_file = '/usercode/news_data.sqlite'
    try:
        with sqlite3.connect(database_file) as connection:
            cursor = connection.cursor()
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS news_table (
                    "Source" VARCHAR(30),
                    "Author Name" TEXT,
                    "News Title" TEXT,
                    "URL" TEXT,
                    "Date Published" TEXT,
                    "Content" TEXT
                )
            ''')
            data.to_sql(name='news_table', con=connection, index=False, if_exists="append")

    except Exception as e:
        logging.error("Error adding item to database: %s", e) 


In [11]:
load_news_data(transformed_df)

### Task 8: Verify Data Loading

In [12]:
try:
    table_name = 'news_table'
    database_path = '/usercode/news_data.sqlite'
    with sqlite3.connect(database_path) as connection:
        cursor = connection.cursor()
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        row_count = cursor.fetchone()[0]
        logging.info(f"Checked table '{table_name}'. It contains {row_count} rows.")
except Exception as e:
        logging.error("Connection Error: %s", e) 

2025-07-27 15:51:26,544 - INFO - Checked table 'news_table'. It contains 397 rows.


In [1]:
# assert len(transformed_df) == row_count, "Discrepancy between size of dataframe and dataset"

### Verify Automated ETL Data Loading:

In [17]:
with sqlite3.connect("/usercode/etl_news_data.sqlite") as connection:
    df = pd.read_sql("SELECT * FROM news_table;", connection)
df.head()

,Source,Author Name,News Title,URL,Date Published,Content
0,Gizmodo.com,Luc Olinga,This Small Town Greek Doctor on How He Uses AI...,https://gizmodo.com/this-small-town-greek-doct...,2025-07-26 14:16:05,"Ioannina, a lakeside town in northern Greece s..."
1,Gizmodo.com,Luc Olinga,The Greek Small Town Doctor Who Knows AI’s Sec...,https://gizmodo.com/the-greek-small-town-docto...,2025-07-26 13:25:28,"On vacation in Greece since July 17, I figured..."
2,Business Insider,Pthompson@Businessinsider.Com (Polly Thompson),I sat in on an AI training session at KPMG. It...,https://www.businessinsider.com/kpmg-ai-traini...,2025-07-26 09:30:01,"On a sweaty Monday morning in June, I joined 9..."
3,Business Insider,Katherine Li,Meta just hired the co-creator of ChatGPT in a...,https://www.businessinsider.com/meta-escalates...,2025-07-26 01:16:36,"Shengjia Zhao, a co-creator of ChatGPT and for..."
4,Slashdot.org,Beauhd,Meta Names Shengjia Zhao As Chief Scientist of...,https://tech.slashdot.org/story/25/07/25/21432...,2025-07-26 00:50:00,Zhao will set a research agenda for MSL under ...
